In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '90 tBodyAccJerk-max()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            nn.Linear(15, 15)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

# Train on Real Test on Real

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_10 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_11 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_12 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_13 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_14 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_15 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9, X_10, X_11, X_12, X_13, X_14, X_15))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9) + [9] * len(X_10) + [10] * len(X_11) + [11] * len(X_12) + [12] * len(X_13) + [13] * len(X_14) + [14] * len(X_15)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [8]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 8.172544479370117, Final Batch Loss: 2.7316579818725586
Epoch 2, Loss: 8.154963731765747, Final Batch Loss: 2.7064859867095947
Epoch 3, Loss: 8.150194644927979, Final Batch Loss: 2.721109390258789
Epoch 4, Loss: 8.146137237548828, Final Batch Loss: 2.7202839851379395
Epoch 5, Loss: 8.13899564743042, Final Batch Loss: 2.712228536605835
Epoch 6, Loss: 8.130512952804565, Final Batch Loss: 2.7118113040924072
Epoch 7, Loss: 8.131357908248901, Final Batch Loss: 2.725954294204712
Epoch 8, Loss: 8.118293523788452, Final Batch Loss: 2.7083590030670166
Epoch 9, Loss: 8.096697092056274, Final Batch Loss: 2.679025650024414
Epoch 10, Loss: 8.099243640899658, Final Batch Loss: 2.6928298473358154
Epoch 11, Loss: 8.09138536453247, Final Batch Loss: 2.6920089721679688
Epoch 12, Loss: 8.081949949264526, Final Batch Loss: 2.6925458908081055
Epoch 13, Loss: 8.074125528335571, Final Batch Loss: 2.6950323581695557
Epoch 14, Loss: 8.050451517105103, Final Batch Loss: 2.6773452758789062
Epoch 1

Epoch 120, Loss: 3.767132878303528, Final Batch Loss: 1.1788127422332764
Epoch 121, Loss: 3.860102415084839, Final Batch Loss: 1.3423594236373901
Epoch 122, Loss: 3.7901018857955933, Final Batch Loss: 1.198707103729248
Epoch 123, Loss: 3.726729154586792, Final Batch Loss: 1.2427669763565063
Epoch 124, Loss: 3.717448353767395, Final Batch Loss: 1.257447600364685
Epoch 125, Loss: 3.760271430015564, Final Batch Loss: 1.2860336303710938
Epoch 126, Loss: 3.76931369304657, Final Batch Loss: 1.1978704929351807
Epoch 127, Loss: 3.7508513927459717, Final Batch Loss: 1.2299760580062866
Epoch 128, Loss: 3.661033272743225, Final Batch Loss: 1.202179193496704
Epoch 129, Loss: 3.7312577962875366, Final Batch Loss: 1.2545747756958008
Epoch 130, Loss: 3.682947039604187, Final Batch Loss: 1.1804968118667603
Epoch 131, Loss: 3.745893359184265, Final Batch Loss: 1.294352412223816
Epoch 132, Loss: 3.623727560043335, Final Batch Loss: 1.3163801431655884
Epoch 133, Loss: 3.6399019956588745, Final Batch Loss

Epoch 234, Loss: 2.8332161903381348, Final Batch Loss: 0.93327796459198
Epoch 235, Loss: 2.7774727940559387, Final Batch Loss: 0.8563329577445984
Epoch 236, Loss: 2.86604380607605, Final Batch Loss: 0.9028850197792053
Epoch 237, Loss: 2.742743670940399, Final Batch Loss: 0.8988276720046997
Epoch 238, Loss: 2.902869999408722, Final Batch Loss: 1.0384595394134521
Epoch 239, Loss: 2.8143069744110107, Final Batch Loss: 0.9495493769645691
Epoch 240, Loss: 2.8119568824768066, Final Batch Loss: 0.8934794068336487
Epoch 241, Loss: 2.8764721751213074, Final Batch Loss: 1.0197991132736206
Epoch 242, Loss: 2.7660643458366394, Final Batch Loss: 0.891944944858551
Epoch 243, Loss: 2.7524913549423218, Final Batch Loss: 0.8946846723556519
Epoch 244, Loss: 2.790008306503296, Final Batch Loss: 0.977592945098877
Epoch 245, Loss: 2.764802634716034, Final Batch Loss: 0.8512153029441833
Epoch 246, Loss: 2.6365898847579956, Final Batch Loss: 0.835214376449585
Epoch 247, Loss: 2.7810025811195374, Final Batch 

Epoch 356, Loss: 2.4448389410972595, Final Batch Loss: 0.9184064269065857
Epoch 357, Loss: 2.4483324885368347, Final Batch Loss: 0.8516925573348999
Epoch 358, Loss: 2.3489471077919006, Final Batch Loss: 0.7277833819389343
Epoch 359, Loss: 2.2969375252723694, Final Batch Loss: 0.7043113112449646
Epoch 360, Loss: 2.3862507343292236, Final Batch Loss: 0.8550324440002441
Epoch 361, Loss: 2.3815714716911316, Final Batch Loss: 0.7526801228523254
Epoch 362, Loss: 2.280105471611023, Final Batch Loss: 0.7295131683349609
Epoch 363, Loss: 2.2767646312713623, Final Batch Loss: 0.7097166776657104
Epoch 364, Loss: 2.278864026069641, Final Batch Loss: 0.6604459881782532
Epoch 365, Loss: 2.3162500858306885, Final Batch Loss: 0.7495490312576294
Epoch 366, Loss: 2.2945291996002197, Final Batch Loss: 0.815077543258667
Epoch 367, Loss: 2.379838287830353, Final Batch Loss: 0.8473616242408752
Epoch 368, Loss: 2.3146650791168213, Final Batch Loss: 0.688154399394989
Epoch 369, Loss: 2.257830262184143, Final B

Epoch 474, Loss: 1.9142173528671265, Final Batch Loss: 0.5282555818557739
Epoch 475, Loss: 2.1625670194625854, Final Batch Loss: 0.7613092660903931
Epoch 476, Loss: 1.9102672338485718, Final Batch Loss: 0.6157441139221191
Epoch 477, Loss: 2.1284517645835876, Final Batch Loss: 0.7180635929107666
Epoch 478, Loss: 2.2188428044319153, Final Batch Loss: 0.7475114464759827
Epoch 479, Loss: 2.1360787749290466, Final Batch Loss: 0.7964493632316589
Epoch 480, Loss: 2.0402934551239014, Final Batch Loss: 0.6016120910644531
Epoch 481, Loss: 2.1094512343406677, Final Batch Loss: 0.7281701564788818
Epoch 482, Loss: 2.1761869192123413, Final Batch Loss: 0.7564082145690918
Epoch 483, Loss: 2.063679277896881, Final Batch Loss: 0.6582834124565125
Epoch 484, Loss: 2.0744786858558655, Final Batch Loss: 0.600000262260437
Epoch 485, Loss: 2.057546854019165, Final Batch Loss: 0.7688714265823364
Epoch 486, Loss: 2.060306966304779, Final Batch Loss: 0.6763064861297607
Epoch 487, Loss: 2.0538359880447388, Final

...

Epoch 4488, Loss: 0.7785325348377228, Final Batch Loss: 0.25793102383613586
Epoch 4489, Loss: 0.7168120592832565, Final Batch Loss: 0.27530989050865173
Epoch 4490, Loss: 0.7096008658409119, Final Batch Loss: 0.17846094071865082
Epoch 4491, Loss: 0.824307307600975, Final Batch Loss: 0.36485156416893005
Epoch 4492, Loss: 0.7981959283351898, Final Batch Loss: 0.26599669456481934
Epoch 4493, Loss: 0.8534530848264694, Final Batch Loss: 0.33836662769317627
Epoch 4494, Loss: 0.7075465768575668, Final Batch Loss: 0.26575836539268494
Epoch 4495, Loss: 0.7902192324399948, Final Batch Loss: 0.3312723636627197
Epoch 4496, Loss: 0.7490160167217255, Final Batch Loss: 0.28913596272468567
Epoch 4497, Loss: 0.724363312125206, Final Batch Loss: 0.30348241329193115
Epoch 4498, Loss: 0.7713930159807205, Final Batch Loss: 0.25084105134010315
Epoch 4499, Loss: 0.8842328637838364, Final Batch Loss: 0.39145320653915405
Epoch 4500, Loss: 0.8319536447525024, Final Batch Loss: 0.2969273626804352
Epoch 4501, Loss

Epoch 4603, Loss: 0.7793871909379959, Final Batch Loss: 0.24719345569610596
Epoch 4604, Loss: 0.7138343155384064, Final Batch Loss: 0.2881453037261963
Epoch 4605, Loss: 0.5806925296783447, Final Batch Loss: 0.18596552312374115
Epoch 4606, Loss: 0.7470577508211136, Final Batch Loss: 0.25367292761802673
Epoch 4607, Loss: 0.8018348217010498, Final Batch Loss: 0.2947711944580078
Epoch 4608, Loss: 0.8330143690109253, Final Batch Loss: 0.2816441059112549
Epoch 4609, Loss: 0.6103885322809219, Final Batch Loss: 0.1696217954158783
Epoch 4610, Loss: 0.7808433473110199, Final Batch Loss: 0.2523907721042633
Epoch 4611, Loss: 0.6392036229372025, Final Batch Loss: 0.164871945977211
Epoch 4612, Loss: 0.6398729830980301, Final Batch Loss: 0.18104201555252075
Epoch 4613, Loss: 0.7834833711385727, Final Batch Loss: 0.3034244179725647
Epoch 4614, Loss: 0.8361353874206543, Final Batch Loss: 0.30908405780792236
Epoch 4615, Loss: 0.7131297141313553, Final Batch Loss: 0.22278675436973572
Epoch 4616, Loss: 0.

Epoch 4714, Loss: 0.7308946996927261, Final Batch Loss: 0.18814577162265778
Epoch 4715, Loss: 0.6463800966739655, Final Batch Loss: 0.23137475550174713
Epoch 4716, Loss: 0.7221604287624359, Final Batch Loss: 0.23130559921264648
Epoch 4717, Loss: 0.8132513463497162, Final Batch Loss: 0.30070972442626953
Epoch 4718, Loss: 0.7444133758544922, Final Batch Loss: 0.24137762188911438
Epoch 4719, Loss: 0.7610114812850952, Final Batch Loss: 0.2518845796585083
Epoch 4720, Loss: 0.8297595530748367, Final Batch Loss: 0.3376579284667969
Epoch 4721, Loss: 0.75572569668293, Final Batch Loss: 0.24043698608875275
Epoch 4722, Loss: 0.81389020383358, Final Batch Loss: 0.3113775849342346
Epoch 4723, Loss: 0.7463622093200684, Final Batch Loss: 0.23644351959228516
Epoch 4724, Loss: 0.7606267333030701, Final Batch Loss: 0.2391050010919571
Epoch 4725, Loss: 0.7234482616186142, Final Batch Loss: 0.2058858722448349
Epoch 4726, Loss: 0.7021554708480835, Final Batch Loss: 0.19174273312091827
Epoch 4727, Loss: 0.8

Epoch 4826, Loss: 0.6537554711103439, Final Batch Loss: 0.14755196869373322
Epoch 4827, Loss: 0.6553096175193787, Final Batch Loss: 0.14792640507221222
Epoch 4828, Loss: 0.7269390970468521, Final Batch Loss: 0.1815422922372818
Epoch 4829, Loss: 0.6711417138576508, Final Batch Loss: 0.1923018991947174
Epoch 4830, Loss: 0.754102349281311, Final Batch Loss: 0.2537227272987366
Epoch 4831, Loss: 0.6906582713127136, Final Batch Loss: 0.1375775933265686
Epoch 4832, Loss: 0.7251473665237427, Final Batch Loss: 0.2782171666622162
Epoch 4833, Loss: 0.6459711790084839, Final Batch Loss: 0.2010246068239212
Epoch 4834, Loss: 0.7085820436477661, Final Batch Loss: 0.26825010776519775
Epoch 4835, Loss: 0.679128959774971, Final Batch Loss: 0.25339028239250183
Epoch 4836, Loss: 0.6646818965673447, Final Batch Loss: 0.14528656005859375
Epoch 4837, Loss: 0.676857978105545, Final Batch Loss: 0.2330850511789322
Epoch 4838, Loss: 0.749723419547081, Final Batch Loss: 0.21671853959560394
Epoch 4839, Loss: 0.669

Epoch 4942, Loss: 0.7496336251497269, Final Batch Loss: 0.26205840706825256
Epoch 4943, Loss: 0.7423306852579117, Final Batch Loss: 0.20664416253566742
Epoch 4944, Loss: 0.7830425947904587, Final Batch Loss: 0.3018244504928589
Epoch 4945, Loss: 0.758674681186676, Final Batch Loss: 0.25853294134140015
Epoch 4946, Loss: 0.8174761235713959, Final Batch Loss: 0.26994240283966064
Epoch 4947, Loss: 0.7215414792299271, Final Batch Loss: 0.19475112855434418
Epoch 4948, Loss: 0.6812148541212082, Final Batch Loss: 0.2039886862039566
Epoch 4949, Loss: 0.6973604559898376, Final Batch Loss: 0.20105914771556854
Epoch 4950, Loss: 0.7070957273244858, Final Batch Loss: 0.22125962376594543
Epoch 4951, Loss: 0.6791283786296844, Final Batch Loss: 0.2526906132698059
Epoch 4952, Loss: 0.7925958633422852, Final Batch Loss: 0.27858489751815796
Epoch 4953, Loss: 0.6533331274986267, Final Batch Loss: 0.2651267945766449
Epoch 4954, Loss: 0.7353804409503937, Final Batch Loss: 0.22510996460914612
Epoch 4955, Loss:

In [9]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[23  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0  0  2  0]
 [ 0  0  7  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  6  0  0  0  0  0  0  0  0  7]
 [ 0  0  0  0  0  0  9  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  5  0  0  0  0  0  1]
 [ 0  1  0  0  0  0  0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0 10  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 10  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  9  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0  6  0]
 [ 0  0  4  0  0  0  0  0  0  0  0  0  0  0  3]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        23
           1    0.85714   0.85714   0.85714        14
           2    0.53846   1.00000   0.70000         7
           3    1.00000   0.85714   0.92308         7
        

# Train on Fake Test on Real

In [10]:
# 20,000 epochs DEFAULT PARAMETERS FOR THRESHOLDS

In [11]:
gen = Generator(z_dim = 108)
load_model(gen, "3 Label 5 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=108, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=30, bias=True)
    (4): Tanh()
  )
)

In [25]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 5)

fake_labels = []

for k in range(size):
    if act_vectors[0][k] == 0 and usr_vectors[0][k] == 0:
        fake_labels.append(0)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 0:
        fake_labels.append(1)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 0:
        fake_labels.append(2)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 1:
        fake_labels.append(3)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 1:
        fake_labels.append(4)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 1:
        fake_labels.append(5)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 2:
        fake_labels.append(6)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 2:
        fake_labels.append(7)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 2:
        fake_labels.append(8)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 3:
        fake_labels.append(9)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 3:
        fake_labels.append(10)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 3:
        fake_labels.append(11)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 4:
        fake_labels.append(12)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 4:
        fake_labels.append(13)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 4:
        fake_labels.append(14)
        
fake_labels = np.asarray(fake_labels)
to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()
#fake_features = gen(to_gen).detach().numpy()

In [26]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5, zero_division = 0))

[[ 8  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 16  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 19  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  3  0]
 [ 0  0  0  0  0  0  0  0 13  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 11  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  4  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  9  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  0  0]
 [ 0  7  0  0  1  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 10]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000         8
           1    0.63158   1.00000   0.77419        12
           2    1.00000   1.00000   1.00000        11
           3    1.00000   1.00000   1.00000        13
        